# Dans ce Notebook, les inputs, donc le sbt va etre en sparse encoding, tandis que le bdd sera en one hot encoding

le one hot encoding c'est lorsque les lettres sont remplacées par des matrice de taille = au nombre de caractere.
Ces matrices sont pleines de  sauf a l'indice du caractere codé, ou il y a un 

le sparse, c'est juste que chaque lettre est remplacé directement par un nombre

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/MyDrive/DATASET/dataset_ai4utg/Generation test sans pretty print/GenerationTest/'

# Lecture des fichiers 

In [ ]:
list_folder = os.listdir(data_dir)
num_folder = len(list_folder)

# Creation des listes 

In [ ]:
from tqdm import tqdm

In [ ]:
bdd = []
sbt = []

for i  in tqdm (range(1000), desc="loading..."):
  path_iteration = data_dir+'/Dir_Fonction_'+str(i+1)
  bdd_path = path_iteration+"/BDD_Fonction_"+str(i+1)+".txt"
  stb_path = path_iteration+"/SBT_Generation_Output/Fonction_"+str(i+1)+"_SBT.txt"
  bdd_txt = open(bdd_path,'r').read()
  sbt_txt = open(stb_path,'r').read()
  bdd.append(bdd_txt)
  sbt.append(sbt_txt)

loading...: 100%|██████████| 1000/1000 [10:36<00:00,  1.57it/s]


# Creation d'un vocabulaire char by char 

In [ ]:
vocab_bdd = {l for word in bdd for l in word}

In [ ]:
vocab_sbt = {l for word in sbt for l in word}

# info pour creation de matrice

In [ ]:
max_len_bdd = max([len(txt) for txt in bdd])
max_len_sbt = max([len(txt) for txt in sbt])
len_vocab_bdd = len(vocab_bdd)
len_vocab_sbt = len(vocab_sbt)

In [ ]:
print('number of samples :', num_folder)
print('max len sequence sbt :', max_len_sbt )
print('max len sequence bdd :',max_len_bdd)
print('len vocab sbt :',len_vocab_sbt)
print('len vocab bdd :',len_vocab_bdd)

number of samples : 1000
max len sequence sbt : 1572
max len sequence bdd : 193
len vocab sbt : 54
len vocab bdd : 49


# association char to ind

In [ ]:
char_to_ind_bdd = {char : ind for ind,char in enumerate(vocab_bdd)}
ind_to_char_bdd = {ind : char for ind,char in enumerate(vocab_bdd)}

In [ ]:
char_to_ind_sbt = {char : ind for ind,char in enumerate(vocab_sbt)}
ind_to_char_sbt = {ind : char for ind,char in enumerate(vocab_sbt)}

# Creation encoder decoder 

In [ ]:
encoder_input_data = np.zeros((num_folder, max_len_sbt, len_vocab_sbt),dtype='float32')
decoder_input_data = np.zeros((num_folder, max_len_bdd, len_vocab_bdd),dtype='float32')
decoder_target_data = np.zeros((num_folder, max_len_bdd, len_vocab_bdd),dtype='float32')

In [ ]:
decoder_input_data.shape

(1000, 193, 49)

In [ ]:
encoder_input_data.shape

(1000, 1572, 54)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(sbt, bdd)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, char_to_ind_sbt[char]] = 1.
    encoder_input_data[i, t + 1:, char_to_ind_sbt[' ']] = 1.
    
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, char_to_ind_bdd[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, char_to_ind_bdd[char]] = 1.            
    decoder_input_data[i, t + 1:, char_to_ind_bdd[' ']] = 1.    
    decoder_target_data[i, t:, char_to_ind_bdd[' ']] = 1.

In [ ]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(1000, 1572, 54)
(1000, 193, 49)
(1000, 193, 49)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, TimeDistributed, LSTM, Bidirectional, Embedding, RepeatVector, Input
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Model

In [ ]:
batch_size = 64  
epochs = 50 
latent_dim = 256  

In [ ]:
encoder_inputs = Input(shape=(None, len_vocab_sbt))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, len_vocab_bdd))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(len_vocab_bdd, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 93s 7s/step - loss: 3.1441 - accuracy: 0.2259 - val_loss: 2.8811 - val_accuracy: 0.2647
Epoch 2/50
13/13 [==============================] - 90s 7s/step - loss: 2.7970 - accuracy: 0.2776 - val_loss: 2.6846 - val_accuracy: 0.2632
Epoch 3/50
13/13 [==============================] - 90s 7s/step - loss: 2.5844 - accuracy: 0.2976 - val_loss: 2.4052 - val_accuracy: 0.3383
Epoch 4/50
13/13 [==============================] - 90s 7s/step - loss: 2.3789 - accuracy: 0.3415 - val_loss: 2.2621 - val_accuracy: 0.3286
Epoch 5/50
13/13 [==============================] - 90s 7s/step - loss: 2.2604 - accuracy: 0.3347 - val_loss: 2.2264 - val_accuracy: 0.3794
Epoch 6/50
13/13 [==============================] - 90s 7s/step - loss: 2.0612 - accuracy: 0.4003 - val_loss: 1.8971 - val_accuracy: 0.4315
Epoch 7/50
13/13 [==============================] - 89s 7s/step - loss: 1.8596 - accuracy: 0.4391 - val_loss: 1.9202 - val_accuracy: 0.4097
Epoch 8/50
13/13 [==

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len_vocab_bdd))
    
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, ind_to_char_bdd[' ']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = ind_to_char_bdd[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_len_bdd):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len_vocab_bdd))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', sbt[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence:  ( MethodDeclaration ( Modifier public  ) Modifier public  ( Modifier static  ) Modifier static  ( SimpleName Fonction_1 ) SimpleName Fonction_1 ( Parameter ( PrimitiveType int ) PrimitiveType int ( SimpleName b ) SimpleName b ) Parameter ( Parameter ( PrimitiveType int ) PrimitiveType int ( SimpleName c ) SimpleName c ) Parameter ( PrimitiveType boolean ) PrimitiveType boolean ( BlockStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarator ( PrimitiveType int ) PrimitiveType int ( SimpleName d ) SimpleName d ( UnaryExpr ( IntegerLiteralExpr 724 ) IntegerLiteralExpr 724 ) UnaryExpr ) VariableDeclarator ) VariableDeclarationExpr ) ExpressionStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarator ( PrimitiveType int ) PrimitiveType int ( SimpleName a ) SimpleName a ( IntegerLiteralExpr 507 ) IntegerLiteralExpr 507 ) VariableDeclarator ) VariableDeclarationExpr ) ExpressionStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarator ( Pri

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7eff44dbd7a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 535, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


-
Input sentence:  ( MethodDeclaration ( Modifier public  ) Modifier public  ( Modifier static  ) Modifier static  ( SimpleName Fonction_11 ) SimpleName Fonction_11 ( Parameter ( PrimitiveType int ) PrimitiveType int ( SimpleName b ) SimpleName b ) Parameter ( Parameter ( PrimitiveType int ) PrimitiveType int ( SimpleName p ) SimpleName p ) Parameter ( PrimitiveType boolean ) PrimitiveType boolean ( BlockStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarator ( PrimitiveType int ) PrimitiveType int ( SimpleName e ) SimpleName e ( UnaryExpr ( IntegerLiteralExpr 139 ) IntegerLiteralExpr 139 ) UnaryExpr ) VariableDeclarator ) VariableDeclarationExpr ) ExpressionStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarator ( PrimitiveType int ) PrimitiveType int ( SimpleName o ) SimpleName o ( IntegerLiteralExpr 93 ) IntegerLiteralExpr 93 ) VariableDeclarator ) VariableDeclarationExpr ) ExpressionStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarator ( Pri

KeyboardInterrupt: ignored

# Conversion en one hot encoding de l'output ( bdd )

# Dimension :  ( nombre fichiers : 1000, nombre max de caractere : 193, nombre de caracteres differents : 49 bdd)

# Initialisation des matrices de One Hot encoding

In [ ]:
#encoded_bdd = np.zeros((number_of_folder,1572,49))
encoded_bdd = np.zeros((number_of_folder,193,49))

NameError: ignored

# Conversion du sbt et bdd en one hot encoding

In [ ]:
for i,bdd_code in tqdm( enumerate(bdd)):
    for j,char in enumerate (bdd_code):
        encoded_bdd[i,j,char_to_ind_bdd[char]] = 1.

In [ ]:
encoded_bdd.shape

In [ ]:
#encoded_bdd = np.expand_dims(encoded_bdd, axis=2)

In [ ]:
encoded_bdd.shape

In [ ]:
trainY = encoded_bdd

In [ ]:
trainY.shape

# Decodage du one hot encoding

In [ ]:
decoded_sequence_bdd = []
encoded_sequence_bdd = encoded_bdd[0]
for i,chars in enumerate(encoded_sequence_bdd):
    for j,char in enumerate(chars):
        if (char==1):
            decoded_sequence_bdd.append(ind_to_char_bdd[j])
decoded_sequence_bdd = ''.join(decoded_sequence_bdd)

In [ ]:
print(decoded_sequence_bdd)

In [ ]:
ind_space_sbt = char_to_ind_sbt[' ']

# Encodage du sbt

# Encodage en char to char but sparse

# dimmension : ( nombre fichiers : 1000, nombre de char max : 1572)

In [ ]:
encoded_sbt_char_sparse = np.full((number_of_folder,1572),ind_space_sbt)

In [ ]:
for i,sbt_code in enumerate(sbt):
    for j,char in enumerate (sbt_code):
        encoded_sbt_char_sparse[i,j] = char_to_ind_sbt[char]

In [ ]:
print(encoded_sbt_char_sparse.shape)

In [ ]:
trainX = encoded_sbt_char_sparse

In [ ]:
trainX.shape

# Decodage en sparse char to char

In [ ]:
def decode_pred_sequence(pred_sequence):
    decoded_sequence = []
    for char in pred_sequence:
        decoded_sequence.append(ind_to_char_bdd[char])
    decoded_sequence = ''.join(decoded_sequence)
    return decoded_sequence

In [ ]:
decoded_sequence_sbt_sparse = []
encoded_sbt = encoded_sbt_char_sparse[0]
for seq in encoded_sbt:
    decoded_sequence_sbt_sparse.append(ind_to_char_sbt[seq])
    
decoded_sequence_sbt_sparse = ''.join(decoded_sequence_sbt_sparse)
print(decoded_sequence_sbt_sparse)

In [ ]:
len(decoded_sequence_sbt_sparse)

# Creation d'un dataset

# X : sbt en sparse

# Y : bdd en one hot encoding

In [ ]:
dataset_sparse = tf.data.Dataset.from_tensor_slices((encoded_sbt_char_sparse,encoded_bdd))

In [ ]:
for input,output in dataset_sparse:
    print('input :',input)
    print('output :',output)

    break

# Creation du Modele

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, TimeDistributed, LSTM, Bidirectional, Embedding, RepeatVector
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
def create_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    # Create the model
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model
 
# Create model
model = create_model(len(vocab_sbt)+1, len(vocab_bdd), 1572, 193, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [ ]:

model.summary()

In [ ]:
model.fit(trainX, trainY,batch_size = 128, epochs=180, validation_split=0.1)

In [ ]:
model.save('v1_char_by_char_target_ohe.h5')

In [ ]:
#from tensorflow.keras.models import load_model
#model = load_model('v1_char_by_char_target_ohe.h5')

# Test

In [ ]:
trainX.shape

In [ ]:
test_input = trainX[0]

In [ ]:
test_input = np.expand_dims(test_input,0)
test_input.shape

In [ ]:
prediction = model.predict(test_input)[0]

In [ ]:
prediction.shape

In [ ]:
prediction[0]

In [ ]:
for i in range(10):
  pred = np.argmax(prediction[i])
  print(pred)


In [ ]:
ind_to_char_bdd[45]

In [ ]:
sequence_prediction = []
for i, char in enumerate(prediction):
  sequence_prediction.append(np.argmax(char))

In [ ]:
sequence_prediction

In [ ]:
decoded_sequence = []
for char in sequence_prediction:
  decoded_sequence.append(ind_to_char_bdd[char])
decoded_sequence = ''.join(decoded_sequence)

In [ ]:
len(decoded_sequence)

In [ ]:
print(decoded_sequence)